## Experimentation Platform

no data given yet, user will specify causal structure with the following further info:
* edges [('cause1', 'effect'), ('cause2', 'effect'), ...]
    - (do we want to somewhere automatically generate suggestions using cond-indeps/correlations of causal hypothesis DAG? can be for either og or simplified DAG)
* which nodes can be simplified to one (ex. if there exists columns injury1, injury2... -> injury_categorical)
* which nodes are interveneable
* what is the chosen outcome variable

Then the user's should note which nodes are construct and what proxy data should be used for those.  
note: need to decide how to do data-fusion (multi-view?) and whether to give freedom to use VAEs or other options.  

For each construct variable, a VAE will be train on it to give concrete function of the consturct.  
User will specify whether to categorize construct variables and how.  
The dashborad will create a data dictionary report (what variables are cts, categorical, summary stats).  

Should fit observational data to causal structure, estimate individual treatment effects, let user experiment with do conditionals on interveneable values.

---

Q: do we keep latent nodes and find range of do-cond, or do we make observable via VAEs and then get fixed-point do-cond estimation.  
note: data would almost always be sparse, so need to carefully chose architecture to estimate interventional data, *check assumptions*.

## side features:
* plot "setting" nodes
* dashoboard front-end

* can either train to fit DAG in the experimentation platform, or use pretrained model with DICE  
Q: where and how do we want DICE in the platform, plausibly it can replace the fitting part, as well as can be used for backtesting
---

## First Try:

In [ ]:
# # generate data_dictionary report for a given dataset
# from data_dict import data_dicti

# # might want to discretesize variables, and quantify categorical variables
# data_dicti(df)

In [111]:
class myDAG:
    def __init__(self, edges):
        self.edges = edges
        self.nodes = list(set([x for y in edges for x in y]))
        self.simplify_to_dict = {}
        self.intrevenable_dict = {}

        # ...
        self.intrevenable_dict = {k: False for k in self.nodes}

    def simplify(self, simplify_from, simplify_to):
        # every element from simplify_from will be mapped in a dictionary to simplify_to
        self.simplify_to_dict = {k: simplify_to for k in simplify_from}
        # now replace in edges
        for i in range(len(self.edges)):
            self.edges[i] = [simplify_to if x in simplify_from else x for x in self.edges[i]]         
        # if repeating edges keep all but one
        self.edges = list(set(tuple(x) for x in self.edges))

    def make_intrevenable(self, intrevenable_nodes):
        for k in self.nodes:
            if k in intrevenable_nodes:
                self.intrevenable_dict[k] = True

    def make_target(self, target_node):
        self.target_node = target_node

    def make_constructs(self, construct_nodes):
        self.construct_nodes = construct_nodes

    def plot_orphans(self):
        # plotting parentless nodes, found automatically from networkx graph
        ... 
    
    def quantify_constructs(self, construct, dataset, loss_function):
        # train VAE on construct and dataset, make customizable loss func
        # VAE because...
        # need to make sure discretisizing and quantifying is done correctly for the type of data and construct
        # i.e. one-hot encoding for categorical variables, etc.
        # representation of categorical data matter. so does discretization of continuous data.

        # eventually want to do data fusion on datasets
        # will return a function that can be used on new data to get outcome
        # after created want to add those features to relevant node in training data
        ...
        # maybe: bayessian hyperparameter optimization?

    # check if all preceading steps were completed:
    ...

    def __str__(self):
        return str(self.edges)

In [114]:
# input:
edges = [('is_senior', 'interactions'),
         ('is_adult', 'interactions'),
         ('is_young', 'interactions'),
         ('sale_calls', 'interactions'),
         ('interactions', 'renewal'),
         ('bugs_faced', 'renewal'),
         ('bugs_faced', 'discount'),
         ('monthly_usage', 'renewal'),
         ('monthly_usage', 'discount'),
         ('consumer_trust', 'renewal'),
         ('discount', 'renewal')]
g = myDAG(edges)
g.simplify(['is_senior', 'is_adult', 'is_young'], 'age_status')
g.make_intrevenable(['sale_calls', 'interactions', 'bugs_faced', 'discount', 'consumer_trust'])
g.make_target('renewal')
g.make_constructs(['consumer_trust'])

In [60]:
# testing:
edges2 = [('age_status', 'interactions'), ('sale_calls', 'interactions'), ('interactions', 'renewal'), ('bugs_faced', 'renewal'), ('bugs_faced', 'discount'), ('monthly_usage', 'renewal'), ('monthly_usage', 'discount'), ('consumer_trust', 'renewal'), ('discount', 'renewal')]
# sort element of g.edges and edges2 and compare
sorted(g.edges) == sorted(edges2)

True

In [ ]:
# if all steps completed and myDAG is completely initialized, 
# can now fetch observational data and 'fit' to DAG
...

# then should be able to predict, intrevene, ask counterfactuals...
# make an interactive dashboard for this